In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
from transformers import BertModel, BertConfig, BertTokenizer, RobertaModel

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
!pip uninstall transformers -y

In [1]:
!pip install transformers==2.11.0

     |████████████████████████████████| 674 kB 900 kB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 5.6 MB/s eta 0:00:01     |███████                         | 1.2 MB 3.8 MB/s eta 0:00:02     |██████████████████████          | 3.9 MB 3.8 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.2.2
    Uninstalling transformers-4.2.2:
      Successfully uninstalled transformers-4.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.0.1 requires transformers<4.3,>=4.1, but you have transformers 2.11.0 which is incompatible.


In [4]:
transformers.__version__

'2.11.0'

In [5]:
df=pd.read_csv('../input/semeval/subtaskB_data_all.csv', index_col=0)
df_dev=pd.read_csv('../input/semeval/subtaskB_dev_data.csv', index_col=0)
df_test=pd.read_csv('../input/semeval/subtaskB_test_data.csv',index_col=0)

In [6]:
df1=pd.read_csv('../input/semeval/subtaskB_answers_all.csv', index_col=0, names = ["id", "Class"])
df1_dev=pd.read_csv('../input/semeval/subtaskB_gold_answers.csv', index_col=0, names = ["id", "Class"])
df1_test=pd.read_csv('../input/semeval/subtaskB_gold_answers 2.csv', index_col=0, names=["id", "Class"])

In [7]:
frames=[df,df1]
result=pd.concat(frames, axis=1)
frames=[df_dev,df1_dev]
result_dev=pd.concat(frames, axis=1)
frames=[df_test,df1_test]
result_test=pd.concat(frames, axis=1)

In [8]:
result

,FalseSent,OptionA,OptionB,OptionC,Class
id,,,,,
0,He poured orange juice on his cereal.,Orange juice is usually bright orange.,Orange juice doesn't taste good on cereal.,Orange juice is sticky if you spill it on the ...,B
1,He drinks apple.,Apple juice are very tasty and milk too,Apple can not be drunk,Apple cannot eat a human,B
2,"Jeff ran 100,000 miles today","100,000 miles is way to long for one person to...","Jeff is a four letter name and 100,000 has six...","100,000 miles is longer than 100,000 km.",A
3,I sting a mosquito,A human is a mammal,A human is omnivorous,A human has not stings,C
4,A giraffe is a person.,Giraffes can drink water from a lake.,A giraffe is not a human being.,.Giraffes usually eat leaves.,B
...,...,...,...,...,...
9995,Mark ate a big bitter cherry pie,Mark is bad at making cherry pie,a cherry pie should be big,a cherry pie should be sweet,C
9996,Gloria wears a cat on her head,a hat cannot be worn on a cat's head,a cat cannot be worn on a person's head,the cat is too heavy to be worn on her head,B
9997,Harry went to the barbershop to have his glass...,a barbershop usually don't provide the service...,a barbershop usually repairs computers instead...,the barbershop lacked the necessary tools to r...,A


In [9]:
result['Label'] = ""
result_dev['Label'] = ""
result_test['Label'] = ""

In [10]:
result.loc[result['Class'] == 'A', 'Label'] = 0
result.loc[result['Class'] == 'B', 'Label'] = 1
result.loc[result['Class'] == 'C', 'Label'] = 2
result_dev.loc[result_dev['Class'] == 'A', 'Label'] = 0
result_dev.loc[result_dev['Class'] == 'B', 'Label'] = 1
result_dev.loc[result_dev['Class'] == 'C', 'Label'] = 2
result_test.loc[result_test['Class'] == 'A', 'Label'] = 0
result_test.loc[result_test['Class'] == 'B', 'Label'] = 1
result_test.loc[result_test['Class'] == 'C', 'Label'] = 2

In [11]:
result=result.drop(index=[5618])
result=result.drop(index=[7238])
result=result.drop(index=[9941])
result=result.reset_index()

In [12]:
result

,id,FalseSent,OptionA,OptionB,OptionC,Class,Label
0,0,He poured orange juice on his cereal.,Orange juice is usually bright orange.,Orange juice doesn't taste good on cereal.,Orange juice is sticky if you spill it on the ...,B,1
1,1,He drinks apple.,Apple juice are very tasty and milk too,Apple can not be drunk,Apple cannot eat a human,B,1
2,2,"Jeff ran 100,000 miles today","100,000 miles is way to long for one person to...","Jeff is a four letter name and 100,000 has six...","100,000 miles is longer than 100,000 km.",A,0
3,3,I sting a mosquito,A human is a mammal,A human is omnivorous,A human has not stings,C,2
4,4,A giraffe is a person.,Giraffes can drink water from a lake.,A giraffe is not a human being.,.Giraffes usually eat leaves.,B,1
...,...,...,...,...,...,...,...
9992,9995,Mark ate a big bitter cherry pie,Mark is bad at making cherry pie,a cherry pie should be big,a cherry pie should be sweet,C,2
9993,9996,Gloria wears a cat on her head,a hat cannot be worn on a cat's head,a cat cannot be worn on a person's head,the cat is too heavy to be worn on her head,B,1
9994,9997,Harry went to the barbershop to have his glass...,a barbershop usually don't provide the service...,a barbershop usually repairs computers instead...,the barbershop lacked the necessary tools to r...,A,0
9995,9998,Reilly is sleeping on the window,the window is open and a person cannot lay on it,the window is too cold to sleep on it,a person cannot sleep on a window,C,2


In [13]:
column_names = ["FalseSent","Option", "Class"]

result1 = pd.DataFrame(columns = column_names)
result1_dev = pd.DataFrame(columns = column_names)
result1_test = pd.DataFrame(columns = column_names)

In [14]:
result1

,FalseSent,Option,Class


In [15]:
falsesent=[]
option=[]
labelname=[]
for i in range(len(result)):
    x=result.iloc[i]['FalseSent']
    a=result.iloc[i]['OptionA']
    b=result.iloc[i]['OptionB']
    c=result.iloc[i]['OptionC']
    l=result.iloc[i]['Label']
    falsesent.append(x)
    falsesent.append(x)
    falsesent.append(x)
    option.append(a)
    option.append(b)
    option.append(c)
    if str(l) == str(0):
        labelname.append(1)
    else:
        labelname.append(0)
    if str(l) == str(1):
        labelname.append(1)
    else:
        labelname.append(0)
    if str(l) == str(2):
        labelname.append(1)
    else:
        labelname.append(0)

In [16]:
result1['FalseSent'] = falsesent
result1['Option'] = option
result1['Class'] = labelname

In [18]:
result1[0:10]

,FalseSent,Option,Class
0,He poured orange juice on his cereal.,Orange juice is usually bright orange.,0
1,He poured orange juice on his cereal.,Orange juice doesn't taste good on cereal.,1
2,He poured orange juice on his cereal.,Orange juice is sticky if you spill it on the ...,0
3,He drinks apple.,Apple juice are very tasty and milk too,0
4,He drinks apple.,Apple can not be drunk,1
5,He drinks apple.,Apple cannot eat a human,0
6,"Jeff ran 100,000 miles today","100,000 miles is way to long for one person to...",1
7,"Jeff ran 100,000 miles today","Jeff is a four letter name and 100,000 has six...",0
8,"Jeff ran 100,000 miles today","100,000 miles is longer than 100,000 km.",0
9,I sting a mosquito,A human is a mammal,0


In [19]:
falsesent=[]
option=[]
labelname=[]
for i in range(len(result_dev)):
    x=result_dev.iloc[i]['FalseSent']
    a=result_dev.iloc[i]['OptionA']
    b=result_dev.iloc[i]['OptionB']
    c=result_dev.iloc[i]['OptionC']
    l=result_dev.iloc[i]['Label']
    falsesent.append(x)
    falsesent.append(x)
    falsesent.append(x)
    option.append(a)
    option.append(b)
    option.append(c)
    if str(l) == str(0):
        labelname.append(1)
    else:
        labelname.append(0)
    if str(l) == str(1):
        labelname.append(1)
    else:
        labelname.append(0)
    if str(l) == str(2):
        labelname.append(1)
    else:
        labelname.append(0)

In [20]:
result1_dev['FalseSent'] = falsesent
result1_dev['Option'] = option
result1_dev['Class'] = labelname

In [21]:
falsesent=[]
option=[]
labelname=[]
for i in range(len(result_test)):
    x=result_test.iloc[i]['FalseSent']
    a=result_test.iloc[i]['OptionA']
    b=result_test.iloc[i]['OptionB']
    c=result_test.iloc[i]['OptionC']
    l=result_test.iloc[i]['Label']
    falsesent.append(x)
    falsesent.append(x)
    falsesent.append(x)
    option.append(a)
    option.append(b)
    option.append(c)
    if str(l) == str(0):
        labelname.append(1)
    else:
        labelname.append(0)
    if str(l) == str(1):
        labelname.append(1)
    else:
        labelname.append(0)
    if str(l) == str(2):
        labelname.append(1)
    else:
        labelname.append(0)

In [22]:
result1_test['FalseSent'] = falsesent
result1_test['Option'] = option
result1_test['Class'] = labelname

In [23]:
result1_test

,FalseSent,Option,Class
0,He loves to stroll at the park with his bed,A bed is too heavy to carry with when strollin...,1
1,He loves to stroll at the park with his bed,walking at a park is good for health,0
2,He loves to stroll at the park with his bed,Some beds are big while some are smaller,0
3,The inverter was able to power the continent.,An inverter is smaller than a car,0
4,The inverter was able to power the continent.,An inverter is incapable of powering an entire...,1
...,...,...,...
2995,The desert has sand that you can drink.,Sand is solid and inedible.,1
2996,The desert has sand that you can drink.,The desert has lots of sand.,0
2997,My friend runs for 2 inches every day.,My friend likes running.,0
2998,My friend runs for 2 inches every day.,2 inches and indicate a distance.,0


In [24]:
y_train = tf.keras.utils.to_categorical(result1.Class, num_classes=2)
y_dev = tf.keras.utils.to_categorical(result1_dev.Class, num_classes=2)
y_test = tf.keras.utils.to_categorical(result1_test.Class, num_classes=2)

In [25]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
epochs = 10

# Labels in our dataset.
labels = [0,1]

In [26]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)


In [27]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    sequence_output, pooled_output = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(64, return_sequences=True))(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(2, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="binary_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Strategy: <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7f9c7212aa10>
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 128, 768), ( 109482240   input_ids[0][0]            

In [28]:
train_data = BertSemanticDataGenerator(
    result1[["FalseSent", "Option"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    result1_dev[["FalseSent", "Option"]].values.astype("str"),
    y_dev,
    batch_size=batch_size,
    shuffle=False,
)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [29]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=25,
)

Epoch 1/10
937/937 [==============================] - 248s 239ms/step - loss: 0.5398 - acc: 0.7290 - val_loss: 0.4853 - val_acc: 0.7534
Epoch 2/10
937/937 [==============================] - 219s 232ms/step - loss: 0.4286 - acc: 0.8001 - val_loss: 0.4327 - val_acc: 0.7907
Epoch 3/10
937/937 [==============================] - 219s 232ms/step - loss: 0.3962 - acc: 0.8164 - val_loss: 0.4368 - val_acc: 0.7890
Epoch 4/10
937/937 [==============================] - 219s 232ms/step - loss: 0.3728 - acc: 0.8304 - val_loss: 0.4195 - val_acc: 0.7981
Epoch 5/10
937/937 [==============================] - 218s 231ms/step - loss: 0.3564 - acc: 0.8411 - val_loss: 0.4428 - val_acc: 0.7933
Epoch 6/10
937/937 [==============================] - 218s 231ms/step - loss: 0.3367 - acc: 0.8522 - val_loss: 0.4561 - val_acc: 0.7964
Epoch 7/10
937/937 [==============================] - 219s 232ms/step - loss: 0.3175 - acc: 0.8600 - val_loss: 0.4483 - val_acc: 0.7970
Epoch 8/10
937/937 [============================

In [30]:
# Unfreeze the bert_model.
bert_model.trainable = True
# Recompile the model to make the change effective.
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_masks (InputLayer)    [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 128, 768), ( 109482240   input_ids[0][0]                  
                                                                 attention_masks[0][0]        

In [31]:
epochs=10
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=25,
)

Epoch 1/10
937/937 [==============================] - 519s 528ms/step - loss: 0.2337 - accuracy: 0.9012 - val_loss: 0.4370 - val_accuracy: 0.8115
Epoch 2/10
937/937 [==============================] - 492s 524ms/step - loss: 0.1705 - accuracy: 0.9319 - val_loss: 0.5025 - val_accuracy: 0.8135
Epoch 3/10
937/937 [==============================] - 494s 525ms/step - loss: 0.1290 - accuracy: 0.9518 - val_loss: 0.5034 - val_accuracy: 0.8159
Epoch 4/10
937/937 [==============================] - 493s 525ms/step - loss: 0.0912 - accuracy: 0.9678 - val_loss: 0.5831 - val_accuracy: 0.8185
Epoch 5/10
937/937 [==============================] - 494s 525ms/step - loss: 0.0653 - accuracy: 0.9778 - val_loss: 0.6078 - val_accuracy: 0.8185
Epoch 6/10
937/937 [==============================] - 493s 524ms/step - loss: 0.0468 - accuracy: 0.9849 - val_loss: 0.6426 - val_accuracy: 0.8189
Epoch 7/10
937/937 [==============================] - 491s 522ms/step - loss: 0.0367 - accuracy: 0.9895 - val_loss: 0.6840 -

In [32]:
test_data = BertSemanticDataGenerator(
    result1_test[["FalseSent", "Option"]].values.astype("str"),
    y_test,
    batch_size=3000,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

1/1 [==============================] - 16s 16s/step - loss: 0.7562 - accuracy: 0.8243


[0.7562215328216553, 0.8243333101272583]

In [33]:
test_pred=model.predict(test_data)

In [34]:
print(len(test_pred))

3000


In [35]:
print(result1_test[3:6])

                                       FalseSent  \
3  The inverter was able to power the continent.   
4  The inverter was able to power the continent.   
5  The inverter was able to power the continent.   

                                              Option  Class  
3                  An inverter is smaller than a car      0  
4  An inverter is incapable of powering an entire...      1  
5                       An inverter is rechargeable.      0  


In [36]:
print(test_pred[3:6])

[[9.9999976e-01 2.5476686e-07]
 [1.9305620e-05 9.9998069e-01]
 [9.9998069e-01 1.9350413e-05]]


In [37]:
print(result_test.iloc[1])

FalseSent        The inverter was able to power the continent.
OptionA                      An inverter is smaller than a car
OptionB      An inverter is incapable of powering an entire...
OptionC                           An inverter is rechargeable.
Class                                                        B
Label                                                        1
Name: 452, dtype: object


In [38]:
predictions = model.predict(test_data)

In [39]:
pred0=[]
pred1=[]
for i in range(len(predictions)):
    pred0.append(predictions[i][0])
    pred1.append(predictions[i][1])

In [40]:
column_names = ["0", "1"]

df = pd.DataFrame(columns = column_names)

In [41]:
df["0"] = pred0
df["1"] = pred1

In [42]:
df.to_csv('comveB.csv')

In [43]:
def func(r):
    if(r['1']>r['0']):
        return [r['1'], 1]
    else:
        return [r['0'], 0]

df['tar'] = df.apply(lambda r: func(r), 1)
df = df.drop(['1','0'], 1)

l = df['tar']

res = []

def func(x, y, z):
    mark = list()
    if(x[1] == 1):
        mark.append(x)
    if(y[1] == 1):
        mark.append(y)
    if(z[1] == 1):
        mark.append(z)
    elif(x[1] == y[1] == z[1] == 0):
        mark.append(x)
        mark.append(y)
        mark.append(z)
        return min(mark)
    return max(mark)

for i in range(0, len(l), 3):
    l[i].append('A')
    l[i+1].append('B')
    l[i+2].append('C')
    res.append(func(l[i], l[i+1], l[i+2]))
    
res

[[0.9998340606689453, 1, 'A'],
 [0.9999806880950928, 1, 'B'],
 [0.9999997615814209, 1, 'C'],
 [0.9496959447860718, 1, 'C'],
 [0.9999996423721313, 1, 'C'],
 [1.0, 0, 'A'],
 [0.9999803304672241, 1, 'B'],
 [0.9999545812606812, 1, 'B'],
 [0.9982713460922241, 1, 'B'],
 [0.8815223574638367, 1, 'C'],
 [0.9999996423721313, 1, 'B'],
 [0.9999990463256836, 1, 'C'],
 [0.9985707998275757, 0, 'A'],
 [0.9933289289474487, 0, 'C'],
 [0.9034208059310913, 0, 'A'],
 [0.9999998807907104, 1, 'C'],
 [0.9999401569366455, 1, 'A'],
 [0.9841188192367554, 1, 'A'],
 [0.9917492866516113, 1, 'B'],
 [1.0, 1, 'B'],
 [0.9999991655349731, 1, 'B'],
 [0.850541889667511, 0, 'C'],
 [1.0, 1, 'C'],
 [0.999997615814209, 0, 'A'],
 [0.9999995231628418, 0, 'C'],
 [0.9981831908226013, 1, 'B'],
 [0.9999997615814209, 1, 'B'],
 [1.0, 1, 'B'],
 [1.0, 1, 'A'],
 [0.9089196920394897, 1, 'A'],
 [1.0, 1, 'A'],
 [1.0, 1, 'A'],
 [0.961788535118103, 0, 'A'],
 [1.0, 1, 'A'],
 [0.9968457818031311, 1, 'B'],
 [0.8267921209335327, 0, 'B'],
 [0.999

In [44]:
print(len(res))

1000


In [45]:
final = pd.DataFrame(res, columns = ['Accuracy', '2', 'Class'])
del final['2']
del final['Accuracy']
final.to_csv('COMVE_b.csv')

In [46]:
final

,Class
0,A
1,B
2,C
3,C
4,C
...,...
995,A
996,C
997,B
998,B


In [47]:
df2 = pd.read_csv('COMVE_b.csv', index_col=0)

In [48]:
df2

,Class
0,A
1,B
2,C
3,C
4,C
...,...
995,A
996,C
997,B
998,B


In [49]:
temp = pd.read_csv('../input/semeval/subtaskB_test_data.csv')

In [50]:
final['in'] = temp['id']

In [51]:
final = final[['in', 'Class']]

In [52]:
final

,in,Class
0,1175,A
1,452,B
2,275,C
3,869,C
4,50,C
...,...,...
995,1114,A
996,8,C
997,1945,B
998,1053,B


In [53]:
final.to_csv('subtaskB_answers.csv', index = True, header = False)